In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import pandasql as psql

In [2]:
# Read candidate master file and name columns
cn_df = pd.read_csv('cn.txt', header=None, sep='|')
cn_df.rename(columns = {cn_df.columns[0]: 'CAND_ID', 
                     cn_df.columns[1]: 'CAND_NAME', 
                     cn_df.columns[2]: 'CAND_PTY_AFFILIATION', 
                     cn_df.columns[3]: 'CAND_ELECTION_YR', 
                     cn_df.columns[4]: 'CAND_OFFICE_ST', 
                     cn_df.columns[5]: 'CAND_OFFICE', 
                     cn_df.columns[6]: 'CAND_OFFICE_DISTRICT', 
                     cn_df.columns[7]: 'CAND_ICI', 
                     cn_df.columns[8]: 'CAND_STATUS', 
                     cn_df.columns[9]: 'CAND_PCC', 
                     cn_df.columns[10]: 'CAND_ST1', 
                     cn_df.columns[11]: 'CAND_ST2', 
                     cn_df.columns[12]: 'CAND_CITY', 
                     cn_df.columns[13]: 'CAND_ST', 
                     cn_df.columns[14]: 'CAND_ZIP'},inplace=True)

# Subset Trump and Biden
q = """
SELECT *
FROM cn_df as d
WHERE CAND_NAME LIKE "Trump%" OR CAND_NAME LIKE "Biden%"
"""
cn_df = psql.sqldf(q)
cn_df

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
0,P80000722,"BIDEN, JOSEPH R JR",DEM,2020,US,P,0.0,C,C,C00703975,PO BOX 58178,None,PHILADELPHIA,PA,19102.0
1,P80001571,"TRUMP, DONALD J.",REP,2020,US,P,0.0,I,C,C00580100,725 FIFTH AVENUE,None,NEW YORK,NY,10022.0


In [3]:
# Read committee master and name columns
cm_df = pd.read_csv('cm.txt', header=None, sep='|')
cm_df.rename(columns = {cm_df.columns[0]: 'CMTE_ID', 
                     cm_df.columns[1]: 'CMTE_NM', 
                     cm_df.columns[2]: 'TRES_NM', 
                     cm_df.columns[3]: 'CMTE_ST1', 
                     cm_df.columns[4]: 'CMTE_ST2', 
                     cm_df.columns[5]: 'CMTE_CITY', 
                     cm_df.columns[6]: 'CMTE_ST', 
                     cm_df.columns[7]: 'CMTE_ZIP', 
                     cm_df.columns[8]: 'CMTE_DSGN', 
                     cm_df.columns[9]: 'CMTE_TP', 
                     cm_df.columns[10]: 'CMTE_PTY_AFFILIATION', 
                     cm_df.columns[11]: 'CMTE_FILING_FREQ', 
                     cm_df.columns[12]: 'ORG_TP', 
                     cm_df.columns[13]: 'CONNECTED_ORG_NM', 
                     cm_df.columns[14]: 'CAND_ID'},inplace=True)
cm_df.head()

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_ID
0,C00000059,HALLMARK CARDS PAC,SARAH MOE,2501 MCGEE,MD #500,KANSAS CITY,MO,64108,U,Q,UNK,M,C,NaN,NaN
1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,"WALKER, KEVIN MR.","25 MASSACHUSETTS AVE, NW",SUITE 600,WASHINGTON,DC,200017400,B,Q,NaN,M,NaN,DELAWARE MEDICAL PAC,NaN
2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,JERRY SIMS JR,3528 W RENO,NaN,OKLAHOMA CITY,OK,73107,U,N,NaN,Q,L,NaN,NaN
3,C00000547,KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE,JERRY SLAUGHTER,623 SW 10TH AVE,NaN,TOPEKA,KS,666121627,U,Q,UNK,Q,M,KANSAS MEDICAL SOCIETY,NaN
4,C00000638,INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...,"ACHENBACH, GRANT MR.","322 CANAL WALK, CANAL LEVEL",NaN,INDIANAPOLIS,IN,46202,U,Q,NaN,T,M,NaN,NaN


In [4]:
# Subset Biden-supporting committees, drop committees against Biden, drop duplicates, reset index
q = """
SELECT *
FROM cm_df as d
WHERE CMTE_NM LIKE "%biden%" OR CONNECTED_ORG_NM LIKE "%biden%" OR CAND_ID = "P80000722" 
"""
biden_df = psql.sqldf(q)
biden_df = biden_df.drop([17])
biden_df = biden_df.drop_duplicates()
biden_df = biden_df.reset_index(drop=True)

In [5]:
# Create array of committee ID's supporting Biden
biden_arr = biden_df['CMTE_ID'].to_numpy()
print(biden_arr)

['C00011197' 'C00703975' 'C00704254' 'C00744946' 'C00745224' 'C00746206'
 'C00746651' 'C00750208' 'C00753186' 'C00754812' 'C00754861' 'C00755264'
 'C00755298' 'C00755660' 'C00755892' 'C00762104' 'C00762229' 'C90019423']


In [6]:
# Subset Trump-supporting committees, drop committees against Trump, drop duplicates, reset index
q = """
SELECT *
FROM cm_df as d
WHERE CMTE_NM LIKE "%trump%" OR CONNECTED_ORG_NM LIKE "%trump%" OR CAND_ID = "P80001571"
"""
trump_df = psql.sqldf(q)
trump_df = trump_df.drop([17, 18, 19, 21, 23, 27, 37, 38])
trump_df = trump_df.drop_duplicates()
trump_df = trump_df.reset_index(drop=True)

In [7]:
trump_arr = trump_df['CMTE_ID'].to_numpy()
print(trump_arr)

['C00003418' 'C00005785' 'C00035618' 'C00040220' 'C00044776' 'C00078196'
 'C00120055' 'C00150672' 'C00153031' 'C00170175' 'C00191288' 'C00580100'
 'C00583716' 'C00618371' 'C00618389' 'C00621672' 'C00628396' 'C00684563'
 'C00696864' 'C00724971' 'C00732230' 'C00733006' 'C00735118' 'C00735258'
 'C00738138' 'C00738310' 'C00738633' 'C00739730' 'C00748616' 'C00749614'
 'C00749937' 'C00752444' 'C00754101' 'C00755686' 'C00760884' 'C00762591'
 'C00763284' 'C00763482' 'C00764787' 'C00785873' 'C90015728' 'C90016668']


In [8]:
# Make an empty dataframe, biden_df with same columns as individual contributions files
dtypes = np.dtype(
    [
        ('CMTE_ID', object),
        ("AMNDT_IND", object),
        ("RPT_TP", object),
        ("TRANSACTION_PGI", object),
        ("IMAGE_NUM", int),
        ("TRANSACTION_TP", object),
        ("ENTITY_TP", object),
        ("NAME", object),
        ("CITY", object),
        ("STATE", object),
        ("ZIP_CODE", object),
        ("EMPLOYER", object),
        ("OCCUPATION", object),
        ("TRANSACTION_DT", int),
        ("TRANSACTION_AMT", int),
        ("OTHER_ID", object),
        ("TRAN_ID", object),
        ("FILE_NUM", int),
        ("MEMO_CD", object),
        ("MEMO_TEXT", object),
        ("SUB_ID", int)
    ]
)
biden_df = pd.DataFrame(np.empty(0, dtype=dtypes))

# Directory path
my_dir = '/Users/darionguan/Desktop/Presidential Prediction/by_date/'
# For file in directory
for filename in os.listdir(my_dir):
    # Create exception for 'invalid_dates.txt'
    if filename.endswith('.txt') is True and filename.endswith('invalid_dates.txt') is False:
        # Set itcontdf with pd.read_csv() but skip lines that have more than 21 columns
        itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')
        # Rename columns
        itcont_df.rename(columns = {itcont_df.columns[0]: 'CMTE_ID', 
                                    itcont_df.columns[1]: 'AMNDT_IND', 
                                    itcont_df.columns[2]: 'RPT_TP', 
                                    itcont_df.columns[3]: 'TRANSACTION_PGI', 
                                    itcont_df.columns[4]: 'IMAGE_NUM', 
                                    itcont_df.columns[5]: 'TRANSACTION_TP', 
                                    itcont_df.columns[6]: 'ENTITY_TP', 
                                    itcont_df.columns[7]: 'NAME', 
                                    itcont_df.columns[8]: 'CITY', 
                                    itcont_df.columns[9]: 'STATE', 
                                    itcont_df.columns[10]: 'ZIP_CODE', 
                                    itcont_df.columns[11]: 'EMPLOYER', 
                                    itcont_df.columns[12]: 'OCCUPATION', 
                                    itcont_df.columns[13]: 'TRANSACTION_DT', 
                                    itcont_df.columns[14]: 'TRANSACTION_AMT', 
                                    itcont_df.columns[15]: 'OTHER_ID', 
                                    itcont_df.columns[16]: 'TRAN_ID', 
                                    itcont_df.columns[17]: 'FILE_NUM', 
                                    itcont_df.columns[18]: 'MEMO_CD', 
                                    itcont_df.columns[19]: 'MEMO_TEXT', 
                                    itcont_df.columns[20]: 'SUB_ID'},inplace=True)
        # working_df = rows of it_contdf where CMTE_ID is in biden_arr
        working_df = itcont_df[itcont_df['CMTE_ID'].isin(biden_arr)]
        # Concatenate to biden_df
        biden_df = pd.concat([biden_df, working_df], ignore_index=True)
# Write as a csv
biden_df.to_csv('biden.csv', index=False)

/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20201028_20201030.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3174677684.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200630_20200711.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3174677684.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200219_20200304.txt


...

b'Skipping line 1117139: expected 21 fields, saw 26\n'
/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3174677684.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200831_20200905.txt


b'Skipping line 1720771: expected 21 fields, saw 26\n'
/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3174677684.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200712_20200723.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3174677684.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


In [9]:
# Make an empty dataframe, trump_df with same columns as individual contributions files
dtypes = np.dtype(
    [
        ('CMTE_ID', object),
        ("AMNDT_IND", object),
        ("RPT_TP", object),
        ("TRANSACTION_PGI", object),
        ("IMAGE_NUM", int),
        ("TRANSACTION_TP", object),
        ("ENTITY_TP", object),
        ("NAME", object),
        ("CITY", object),
        ("STATE", object),
        ("ZIP_CODE", object),
        ("EMPLOYER", object),
        ("OCCUPATION", object),
        ("TRANSACTION_DT", int),
        ("TRANSACTION_AMT", int),
        ("OTHER_ID", object),
        ("TRAN_ID", object),
        ("FILE_NUM", int),
        ("MEMO_CD", object),
        ("MEMO_TEXT", object),
        ("SUB_ID", int)
    ]
)
trump_df = pd.DataFrame(np.empty(0, dtype=dtypes))

# Directory path
my_dir = '/Users/darionguan/Desktop/Presidential Prediction/by_date/'
# For file in directory
for filename in os.listdir(my_dir):
    # Create exception for 'invalid_dates.txt'
    if filename.endswith('.txt') is True and filename.endswith('invalid_dates.txt') is False:
        # Set itcontdf with pd.read_csv() but skip lines that have more than 21 columns
        itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')
        # Rename columns
        itcont_df.rename(columns = {itcont_df.columns[0]: 'CMTE_ID', 
                                    itcont_df.columns[1]: 'AMNDT_IND', 
                                    itcont_df.columns[2]: 'RPT_TP', 
                                    itcont_df.columns[3]: 'TRANSACTION_PGI', 
                                    itcont_df.columns[4]: 'IMAGE_NUM', 
                                    itcont_df.columns[5]: 'TRANSACTION_TP', 
                                    itcont_df.columns[6]: 'ENTITY_TP', 
                                    itcont_df.columns[7]: 'NAME', 
                                    itcont_df.columns[8]: 'CITY', 
                                    itcont_df.columns[9]: 'STATE', 
                                    itcont_df.columns[10]: 'ZIP_CODE', 
                                    itcont_df.columns[11]: 'EMPLOYER', 
                                    itcont_df.columns[12]: 'OCCUPATION', 
                                    itcont_df.columns[13]: 'TRANSACTION_DT', 
                                    itcont_df.columns[14]: 'TRANSACTION_AMT', 
                                    itcont_df.columns[15]: 'OTHER_ID', 
                                    itcont_df.columns[16]: 'TRAN_ID', 
                                    itcont_df.columns[17]: 'FILE_NUM', 
                                    itcont_df.columns[18]: 'MEMO_CD', 
                                    itcont_df.columns[19]: 'MEMO_TEXT', 
                                    itcont_df.columns[20]: 'SUB_ID'},inplace=True)
        # working_df = rows of it_contdf where CMTE_ID is in trump_arr
        working_df = itcont_df[itcont_df['CMTE_ID'].isin(trump_arr)]
        # Concatenate to trump_df
        trump_df = pd.concat([trump_df, working_df], ignore_index=True)
# Set as csv
trump_df.to_csv('trump.csv', index=False)

/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20201028_20201030.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3673036561.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200630_20200711.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3673036561.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200219_20200304.txt


...

b'Skipping line 1117139: expected 21 fields, saw 26\n'
/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3673036561.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200831_20200905.txt


b'Skipping line 1720771: expected 21 fields, saw 26\n'
/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3673036561.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')


/Users/darionguan/Desktop/SAS Project 1/by_date/itcont_2020_20200712_20200723.txt


/var/folders/k1/z3v9vvcx2zj8knzj63hcxqh40000gn/T/ipykernel_92657/3673036561.py:33: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  itcont_df = pd.read_csv(my_dir + filename, header=None, sep='|', on_bad_lines='warn')
